In [2]:
def get_tweet_features_per_row(row):
        #USER features
#         try:
        user_id = row['user.id_str_h']
        user_screen_name = row['user.screen_name_h']
        user_tz = row['user.time_zone']
        user_statuses_count = row['user.statuses_count']
        user_location = row['user.location']
        user_listed_count = row['user.listed_count']
        user_lang = row['user.lang']
        user_friends_count = row['user.friends_count']
        user_followers_count = row['user.followers_count'] # these are other users that the user follows
        user_following = row['user.following'] # this is mostly None
        user_favourites_count = row['user.favourites_count']
        user_description_m = row['user.description_m']
        user_created_at = row['user.created_at'] # use this to later create feature called twitter_age of user for any tweet
        # TWEET features
        id_str_h = row['id_str_h']
        lang = row['lang']
        place = row['place']
#         place_name = row['place.full_name']
        content_language = row['content_language']
        created_at = row['created_at']
        hashtags = row['entities.hashtags']
        tweet_has_media = False if pd.isna(row['entities.media']) else True
        symbols = row['entities.symbols']
        tweet_has_symbol = True if len(row['entities.symbols']) > 0 else False
        user_mentions = row['entities.user_mentions']
        tweet_has_user_mentions = True if len(row['entities.user_mentions']) > 0 else False
        extension_created_dow = row['extension.created_dow']
        extension_created_hod = row['extension.created_hod']
#         retweeted_status_id_str_h = row['retweeted_status.id_str_h']
        retweeted_status_user_id_str_h = row['retweeted_status.user.id_str_h']
        quoted_status_id_str_h = row['quoted_status.id_str_h']
        quoted_status_user_id_str_h = row['quoted_status.user.id_str_h']
        text_m = row['text_m']
        timestamp_ms = row['timestamp_ms']
        retweeted_status_text_m = row['retweeted_status.text_m']
#         except (RuntimeError, TypeError, NameError, ValueError):
#             print("error in evaluation")
        return user_id, user_screen_name, user_tz, user_statuses_count, user_location, user_listed_count, user_lang, user_friends_count, user_followers_count, user_following, user_favourites_count, user_description_m, user_created_at, id_str_h, lang, place, content_language, created_at, hashtags, tweet_has_media, symbols, tweet_has_symbol, user_mentions, tweet_has_user_mentions, extension_created_dow, extension_created_hod, retweeted_status_user_id_str_h, quoted_status_id_str_h, quoted_status_user_id_str_h, text_m, timestamp_ms, retweeted_status_text_m


def get_selected_columns_for_df(twitter_collection_dataframe_name):
    df = pd.DataFrame()
    df['user_id'], df['user_screen_name'], df['user_tz'], df['user_statuses_count'], df['user_location'], df['user_listed_count'], df['user_lang'], df['user_friends_count'], df['user_followers_count'], df['user_following'], df['user_favourites_count'], df['user_description_m'], df['user_created_at'], df['id_str_h'], df['lang'], df['place'], df['content_language'], df['created_at'], df['hashtags'], df['tweet_has_media'], df['symbols'], df['tweet_has_symbol'], df['user_mentions'], df['tweet_has_user_mentions'], df['extension_created_dow'], df['extension_created_hod'], df['retweeted_status_user_id_str_h'], df['quoted_status_id_str_h'], df['quoted_status_user_id_str_h'], df['text_m'], df['timestamp_ms'], df['retweeted_status_text_m']= zip(*twitter_collection_dataframe_name.apply(lambda row: get_tweet_features_per_row(row), axis = 1))
    # Retweet related 
    df['retweeted_status_user_id_str_h'] = df['retweeted_status_user_id_str_h'].fillna(value = 0)
    df['retweet_flag'] = df.apply(lambda r: False if r['retweeted_status_user_id_str_h'] == 0 else True, axis = 1)
    # Quoted Tweet related
    df['quoted_status_id_str_h'] = df['quoted_status_id_str_h'].fillna(value = 0)
    df['quoted_status_user_id_str_h'] = df['quoted_status_user_id_str_h'].fillna(value = 0)
    df['quoted_status_flag'] = df.apply(lambda r: False if r['quoted_status_id_str_h'] == 0 else True, axis = 1)
    print("final data frame created for the given collection with shape: {}".format(df.shape))
    return df

### Data unzipping and collecting JSON in mongo db

In [ ]:
def db_exists(db_name):
    """
    Returns True if database with name db_name exits.
    :param db_name: database name
    :return: True if database with name db_name exits.
    """
    dbnames = mongo_client.list_database_names()
    if db_name in dbnames:
        return True
    else:
        return False


def create_db(db_name):
    """
    Creates a new database with a given name. It create a test collection 'test' with one record ['data':'test'] to
    make sure that new database is instantiated in the mongodb.
    :param db_name: new database name
    :return: True if data base was created successfully
    """
    new_db = mongo_client[db_name]
    test_collection = new_db["test"]
    x = test_collection.insert_one({'data': 'test'})
    if verbose_mode_on:
        print("New database created, test collection added. " + str(x.inserted_id))

    return True


def delete_db(db_name):
    """
    Removes database with a given name.
    :param db_name: database name
    :return: None
    """
    mongo_client.drop_database(db_name)

    if verbose_mode_on:
        print("Database " + str(db_name) + " was dropped.")

    return


### find preferred authors for each user in a given data (of a given duration)

In [ ]:
# def get_preferred_user_for_user(data_to_use):
#     users_distinct = data_to_use[]
#     pass